# Part I. ETL Pipeline for Pre-Processing the Files

## RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## The event_datafile_new.csv contains the following columns: 

- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing the Apache Cassandra connection code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance 

from cassandra.cluster import Cluster
try:
    cluster = Cluster()

    # To establish connection and begin executing queries
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [7]:
# Creating the Sparkify Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# Setting the KEYSPACE to the Sparkify
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## We are going to create tables to answer the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




__________________________________________________________________________________________________________________________________________


## The first QUERY 1 will be:

SELECT artist_name, song_title, songs_lenght FROM songs_data WHERE session_id = 338 AND item_in_session = 4

This query requires the following fields from the event_datafile:
- artist, song_title, songs_length, using the sessionId and itemInSession as the WHERE clause.

So, our table will need this five attributes, and the PRIMARY KEY going to be a COMPOSITE KEY of the sessionId as the PARTITION KEY, and the itemInSession as the Clustering Column.

In [11]:
query = "CREATE TABLE IF NOT EXISTS songs_data "
query = query + "(session_id int,  item_in_session int, artist_name text, song_title text, songs_lenght float, \
                    PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [10]:
# DROP STATEMENT IN CASE WE NEED TO RECREATE THE songs_data TABLE

drop_query = "DROP TABLE IF EXISTS songs_data"
try:
    session.execute(drop_query)
except Exception as e:
    print(e)

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_data (session_id, item_in_session, artist_name, song_title, songs_lenght)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Doing the SELECT to verify our query

In [13]:
query = "select artist_name, song_title, songs_lenght from songs_data WHERE session_id=338 AND item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.songs_lenght)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


__________________________________________________________________________________________________________________________________________


## The second QUERY 2 will be:

SELECT artist_name, song_title, first_name, last_name FROM users_data WHERE user_id = 10 AND session_id = 182

This query requires the following fields from the event_datafile:
- artist_name, song_title, first_name, last_name, using the user_id and sessionId as the WHERE clause and itemInSession as the sorted by.

So, our table will need this seven attributes, and the PRIMARY KEY going to be a COMPOSITE KEY of the user_id and the session_id as the PARTITION KEY, and the itemInSession as the clustering column order for the proposite of the sorted results.

In [16]:
query = "CREATE TABLE IF NOT EXISTS users_data "
query = query + "(session_id int,  item_in_session int, artist_name text, song_title text, user_id int, \
                    first_name text, last_name text, \
                    PRIMARY KEY ((user_id,session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [15]:
# DROP STATEMENT IN CASE WE NEED TO RECREATE THE users_data TABLE

drop_query = "DROP TABLE IF EXISTS users_data"
try:
    session.execute(drop_query)
except Exception as e:
    print(e)

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_data (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Doing the SELECT to verify our query

In [18]:
query = "SELECT artist_name, song_title, first_name, last_name, item_in_session FROM users_data WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.first_name, row.last_name, row.item_in_session)

Down To The Bone Keep On Keepin' On Sylvie Cruz 0
Three Drives Greece 2000 Sylvie Cruz 1
Sebastien Tellier Kilometer Sylvie Cruz 2
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 3


__________________________________________________________________________________________________________________________________________


## The third QUERY 3 will be:

SELECT first_name, last_name, user_id FROM music_app_history WHERE song_title = 'All Hands Against His Own'

This query requires the following fields from the event_datafile:
- song_title, first_name, last_name, using the song_title as the WHERE clause and the userId column to identify the users, because we can have different userId (different person) with the same first and last name, listening to the same song

So, our table will need this four attributes, and the PRIMARY KEY going to be a COMPOSITE KEY of the song_title as the PARTITION KEY and the user_id as the Clustering Column

In [21]:
query = "CREATE TABLE IF NOT EXISTS music_app_history "
query = query + "(song_title text, user_id int, \
                    first_name text, last_name text, \
                    PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [20]:
# DROP STATEMENT IN CASE WE NEED TO RECREATE THE music_app_history TABLE

drop_query = "DROP TABLE IF EXISTS music_app_history"
try:
    session.execute(drop_query)
except Exception as e:
    print(e)

In [22]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_history (user_id, song_title, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (int(line[10]), line[9], line[1], line[4]))

#### Doing the SELECT to verify our query

In [26]:
query = "SELECT first_name, last_name, user_id FROM music_app_history WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name, row.user_id)

Jacqueline Lynch 29
Tegan Levine 80
Sara Johnson 95


### Drop the tables before closing out the sessions

In [27]:
# Droping the songs_data table (first query)

drop_query = "DROP TABLE IF EXISTS songs_data"
try:
    session.execute(drop_query)
except Exception as e:
    print(e)

# Droping the users_data table (second query)

drop_query = "DROP TABLE IF EXISTS users_data"
try:
    session.execute(drop_query)
except Exception as e:
    print(e)

# Droping the music_app_history table (third query)

drop_query = "DROP TABLE IF EXISTS music_app_history"
try:
    session.execute(drop_query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [28]:
session.shutdown()
cluster.shutdown()